In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
import functools
import numpy as np
import pandas as pd

from scipy.special import expit

import sklearn as sk
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from tensorflow import keras
import tensorflow as tf

def split_companies_train_dev_test(companies):
    "Return train, dev, test set for companies"
    train, test = train_test_split(companies, test_size=0.1, stratify = companies.sector)
    train, dev = train_test_split(train, test_size=0.1, stratify = train.sector)
    return train, dev, test


def filter_stocks(stocks, tickers):
    return stocks.loc[tickers]


def df_to_ts(df):
    res = df.copy()
    res.index = pd.DatetimeIndex(pd.to_datetime(res.date))
    res.drop('date', axis=1)
    return res


def log_softmax(x):
    return x - np.log(np.sum(np.exp(x)))


def sigmoid(x):
    return expit(x)


def sample_correlation(df, window_size=63):
    idx = np.random.randint(0, df.shape[0]-window_size)
    ts = df[idx:idx+window_size]
    fmap = lambda s: ts['pct_return'].corr(ts[s])
    indices = ts.columns.tolist()[1:]
    correlations = np.array(list(map(fmap, indices)))
    return correlations


def create_correlation_score(df, sample_size=1):
    res = np.array([log_softmax(sample_correlation(df)/0.05)
                    for i in range(sample_size)])
    return np.exp(np.nanmean(res, 0))


def load_data(stock_filename=None, indices_filename=None):

    if stock_filename is None:
        stock_filename = '../../data/processed/wiki_stocks_returns.csv'

    if indices_filename is None:
        indices_filename = '../../data/processed/wiki_indices_returns.csv'

    stocks = pd.read_csv(stock_filename, index_col=False) # long format
    indices = pd.read_csv(indices_filename, index_col=False) # wide format

    # Implementation of hierarchical clustering
    drop_column = lambda df,i=0: df.drop(df.columns[i], axis=1)

    stocks = drop_column(stocks)
    stocks = stocks.drop('name', axis=1)
    stocks = stocks.dropna()

    companies = stocks.groupby('ticker').first().reset_index()
    sectors_counts = companies.sector.value_counts()
    sectors_proportions = sectors_counts/sectors_counts.sum()
    sectors_unique = sectors_counts.index.tolist()

    stocks = stocks.set_index('ticker')

    indices_ts = df_to_ts(indices[['date'] + sectors_unique])
    stocks_ts = df_to_ts(stocks.reset_index())

    stocks_all = pd.merge(stocks_ts, indices_ts, 'left')
    stocks_all = stocks_all.dropna() # loss of 200 000 observations
    stocks_all = stocks_all.drop('sector', axis=1)
    stocks_all = stocks_all.groupby('ticker').apply(df_to_ts)
    stocks_all = stocks_all.drop(['ticker', 'date'], axis=1)
    stocks_all = stocks_all.rename(columns={'close': 'pct_return'})

    label_encoder = preprocessing.LabelEncoder()
    label_encoder.fit(sectors_counts.index.tolist())
    ticker_to_sector = dict(zip(companies.ticker, label_encoder.transform(companies.sector)))

    return stocks_all, companies, label_encoder, ticker_to_sector

def sectors_statistics(companies):
    sectors_counts = companies.sector.value_counts()
    sectors_proportions = sectors_counts/sectors_counts.sum()
    sectors_unique = sectors_counts.index.tolist()
    return sectors_counts, sectors_proportions, sectors_unique


def add_common_layers(y):
    y = keras.layers.BatchNormalization()(y)
    y = keras.layers.LeakyReLU()(y)
    return y


def grouped_convolution(y, nb_channels, _strides, cardinality=4):
    # when `cardinality` == 1 this is just a standard convolution
    if cardinality == 1:
        return keras.layers.Conv1D(nb_channels, kernel_size=10, strides=_strides, padding='same')(y)

    assert not nb_channels % cardinality
    _d = nb_channels // cardinality

    # in a grouped convolution layer, input and output channels are divided into `cardinality` groups,
    # and convolutions are separately performed within each group
    groups = []
    for j in range(cardinality):
        group = keras.layers.Lambda(lambda z: z[:, :, j * _d:j * _d + _d])(y)
        groups.append(keras.layers.Conv1D(_d, kernel_size=10, strides=_strides, padding='same')(group))

    # the grouped convolutional layer concatenates them as the outputs of the layer
    y = keras.layers.concatenate(groups)

    return y


def residual_block(y, nb_channels_in, nb_channels_out, cardinality=4, _strides=1, _project_shortcut=False):
    """
    Our network consists of a stack of residual blocks. These blocks have the same topology,
    and are subject to two simple rules:
    - If producing spatial maps of the same size, the blocks share the same hyper-parameters (width and filter sizes).
    - Each time the spatial map is down-sampled by a factor of 2, the width of the blocks is multiplied by a factor of 2.
    """
    shortcut = y
    kl = keras.layers
    # we modify the residual building block as a bottleneck design to make the network more economical
    y = kl.Conv1D(nb_channels_in, kernel_size=1, strides=1, padding='same')(y)
    y = add_common_layers(y)

    # ResNeXt (identical to ResNet when `cardinality` == 1)
    y = grouped_convolution(y, nb_channels_in, _strides=_strides)
    y = add_common_layers(y)

    y = kl.Conv1D(nb_channels_out, kernel_size=1, strides=1, padding='same')(y)
    # batch normalization is employed after aggregating the transformations and before adding to the shortcut
    y = kl.BatchNormalization()(y)

    # identity shortcuts used directly when the input and output are of the same dimensions
    if _project_shortcut or _strides != 1:
        # when the dimensions increase projection shortcut is used to match dimensions (done by 1×1 convolutions)
        # when the shortcuts go across feature maps of two sizes, they are performed with a stride of 2
        shortcut = kl.Conv1D(nb_channels_out, kernel_size=1, strides=_strides, padding='same')(shortcut)
        shortcut = kl.BatchNormalization()(shortcut)

    y = kl.add([shortcut, y])

    # relu is performed right after each batch normalization,
    # expect for the output of the block where relu is performed after the adding to the shortcut
    y = kl.LeakyReLU()(y)

    return y



# reparameterization trick
# instead of sampling from Q(z|X), sample eps = N(0,I)
# z = z_mean + sqrt(var)*eps
def sampling(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.
    # Arguments:
        args (tensor): mean and log of variance of Q(z|X)
    # Returns:
        z (tensor): sampled latent vector
    """
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = keras.backend.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = tf.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

class CovarianceLayer(keras.layers.Layer):

    def __init__(self, num_classes, **kwargs):
        self.num_classes = num_classes
        super(CovarianceLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        super(CovarianceLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs):
        series_input, environment_input = inputs
        series_input_multiple = tf.tile(series_input, [1, 1, self.num_classes])
        covariances = tf.reduce_mean(series_input_multiple * environment_input, axis=1);
        return covariances

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


def random_subset(df, window_size=21):
    idx = np.random.randint(0, df.shape[0]-window_size)
    ts = df[idx:idx+window_size]
    return ts


def make_keras_subset(dataset_type, companies_data, stocks_data, label_encoder, batch_size, window_size=21):
    idx = np.random.choice(companies_data[dataset_type].shape[0], batch_size)
    df = companies_data[dataset_type].iloc[idx]

    model_input_data = [random_subset(stocks_data[dataset_type].loc[t], window_size) for t in df.ticker]
    model_series_input = np.array([df['pct_return'].values for df in model_input_data])
    model_series_input = model_series_input.reshape(-1, window_size, 1)

    model_environment_input = np.array([df.iloc[:, 1:].values for df in model_input_data])

    y_true = label_encoder.transform(df.sector)

    return model_series_input, model_environment_input, y_true


class StocksSequence(keras.utils.Sequence):

    def __init__(self, stocks_data,  companies_data, window_size, label_encoder, batch_size):
        self.stocks_data = stocks_data
        self.batch_size = batch_size
        self.label_encoder = label_encoder
        self.companies_data = companies_data
        self.window_size = window_size

    def __len__(self):
        return int(np.ceil(self.stocks_data.shape[0] / float(self.batch_size)))

    def __getitem__(self, idx):

        idx = np.random.choice(self.companies_data.shape[0], self.batch_size)
        df = self.companies_data.iloc[idx]
        model_input_data = [random_subset(self.stocks_data.loc[t], self.window_size) for t in df.ticker]
        model_series_input = np.array([df['pct_return'].values for df in model_input_data])
        model_series_input = model_series_input.reshape(-1, self.window_size, 1)
        model_environment_input = np.array([df.iloc[:, 1:].values for df in model_input_data])
        y_true = self.label_encoder.transform(df.sector)

        return [model_series_input, model_environment_input], y_true





In [3]:
?keras.layers.Dense

In [4]:

def make_model(num_classes=16, window_size=21, latent_dim=32):
    kl = keras.layers

    series_input = keras.layers.Input(shape=(window_size, 1), dtype='float32', name='series_input')
    environment_input = kl.Input(shape=(window_size, num_classes), dtype='float32', name='environment_input')
    x = kl.Concatenate()([series_input, environment_input])
    tanh_layer = kl.Lambda(lambda y: tf.tanh(tf.scalar_mul(5, y)))
    x_tanh = tanh_layer(x)
    x = kl.Conv1D(16, 5, 1, activation='relu')(x_tanh)
    x = kl.Dropout(0.01)(x)
    x = residual_block(x, 16, 16, 2)
    x = kl.MaxPool1D()(x)
    x = kl.Conv1D(32, 21, 1, activation='relu')(x)
    x = kl.Dropout(0.01)(x)
    x = residual_block(x, 32, 32, 2)
    x = kl.MaxPool1D()(x)
    
    
    covariances = CovarianceLayer(num_classes)([tanh_layer(series_input), tanh_layer(environment_input)])
    covariances = kl.Dense(16, activation='relu')(covariances)
    covariances = kl.Dense(32, activation='relu')(covariances)

    x = kl.Flatten()(x)
    x = kl.Dense(32, 'relu', kernel_regularizer=keras.regularizers.l2(0.01))(x)
    x = kl.Dropout(0.01)(x)
    z = kl.Multiply(name='Embedding')([x, covariances])
    z = kl.Dense(32, 'relu', kernel_regularizer=keras.regularizers.l2(0.01))(z)
    x_pred = kl.Dense(num_classes, 'softmax')(z)

    model = keras.Model(inputs = [series_input, environment_input], outputs=[x_pred], name='Classifier')

    # kl_batch = - .5 * tf.reduce_sum(1 + x_log_var - tf.square(x_mu) - tf.exp(x_log_var), axis=-1)
    # model.add_loss(kl_batch)

    return model

In [5]:
# Make train dev test set.
np.random.seed(42)

### Feature engineering

stock_filename = '../data/processed/wiki_stocks_returns.csv'
indices_filename = '../data/processed/wiki_indices_returns.csv'

stocks_all, companies, label_encoder, ticker_to_sector = load_data(stock_filename, indices_filename)
sectors_counts, sectors_proportions, sectors_unique = sectors_statistics(companies)

max_proportion_baseline = sectors_proportions.max()
biggest_sector = sectors_proportions.argmax()

print("Most representated class:", biggest_sector, ', with proportion of ', round(100*max_proportion_baseline, 2), '%.')
# Accuracy of our models should be better than max_proportion_baseline.

companies_data = {}
data_split = split_companies_train_dev_test(companies)
for i, k in enumerate(['train', 'dev', 'test']):
    companies_data[k] = data_split[i]
stocks_data = {k: filter_stocks(stocks_all, v.ticker) for k, v in companies_data.items()}


Most representated class: Financial Services , with proportion of  13.09 %.


In [6]:
window_size = 126
model = make_model(window_size=window_size)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [7]:
if False:
    model.load_weights('checkpoint/model_weights_second.json')

In [8]:
import math
def schedule_fn(epoch):
   initial_lrate = 0.001
   drop = 0.5
   epochs_drop = 20.0
   lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
   return lrate

In [9]:
batch_size = 64

optimizer = keras.optimizers.Adam(0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

callbacks = [
    keras.callbacks.ModelCheckpoint('checkpoint/model_weights_eighth.json', monitor='val_acc', verbose=1, save_best_only=True, mode='max'),
    keras.callbacks.TensorBoard(log_dir='./logs/eighth'),
    keras.callbacks.LearningRateScheduler(schedule_fn)
]

stocks_sequence_training = StocksSequence(stocks_data['train'], companies_data['train'], window_size, label_encoder, batch_size)
stocks_sequence_validation = StocksSequence(stocks_data['dev'], companies_data['dev'], window_size, label_encoder, batch_size)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
series_input (InputLayer)       (None, 126, 1)       0                                            
__________________________________________________________________________________________________
environment_input (InputLayer)  (None, 126, 16)      0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 126, 17)      0           series_input[0][0]               
                                                                 environment_input[0][0]          
______________________________________________________________

dense_4 (Dense)                 (None, 32)           1056        Embedding[0][0]                  
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 16)           528         dense_4[0][0]                    
Total params: 41,552
Trainable params: 41,264
Non-trainable params: 288
__________________________________________________________________________________________________
None


In [ ]:
model.fit_generator(
    stocks_sequence_training, steps_per_epoch=2000, epochs=500, callbacks=callbacks,
    validation_data = stocks_sequence_validation, validation_steps=100, workers=4, max_queue_size=20, verbose=1,
    use_multiprocessing=True)

stocks_sequence_test = StocksSequence(stocks_data['test'], companies_data['test'], window_size, label_encoder, batch_size)

model.evaluate_generator(stocks_sequence_test, 200, workers=2, use_multiprocessing=True)

Epoch 1/500
2000/2000 [==============================]2000/2000 [==============================] - 225s 112ms/step - loss: 2.3777 - acc: 0.2512 - val_loss: 2.8392 - val_acc: 0.2222

Epoch 2/500
2000/2000 [==============================]2000/2000 [==============================] - 171s 85ms/step - loss: 2.0545 - acc: 0.3524 - val_loss: 2.8178 - val_acc: 0.2013

Epoch 3/500
2000/2000 [==============================]2000/2000 [==============================] - 204s 102ms/step - loss: 1.9415 - acc: 0.3948 - val_loss: 2.6065 - val_acc: 0.2978

Epoch 4/500
2000/2000 [==============================]2000/2000 [==============================] - 254s 127ms/step - loss: 1.8161 - acc: 0.4269 - val_loss: 2.1902 - val_acc: 0.3508

Epoch 5/500
2000/2000 [==============================]2000/2000 [==============================] - 210s 105ms/step - loss: 1.7522 - acc: 0.4469 - val_loss: 2.8784 - val_acc: 0.2687

Epoch 6/500
2000/2000 [==============================]2000/2000 [==========================

2000/2000 [==============================]2000/2000 [==============================] - 263s 132ms/step - loss: 1.4348 - acc: 0.5474 - val_loss: 2.0350 - val_acc: 0.4322

Epoch 24/500
2000/2000 [==============================]2000/2000 [==============================] - 262s 131ms/step - loss: 1.4446 - acc: 0.5466 - val_loss: 2.1018 - val_acc: 0.4106

Epoch 25/500
2000/2000 [==============================]2000/2000 [==============================] - 261s 130ms/step - loss: 1.4384 - acc: 0.5479 - val_loss: 1.9543 - val_acc: 0.4523

Epoch 26/500
2000/2000 [==============================]2000/2000 [==============================] - 250s 125ms/step - loss: 1.4351 - acc: 0.5511 - val_loss: 2.2123 - val_acc: 0.3955

Epoch 27/500
2000/2000 [==============================]2000/2000 [==============================] - 256s 128ms/step - loss: 1.4082 - acc: 0.5609 - val_loss: 2.2626 - val_acc: 0.4194

Epoch 28/500
2000/2000 [==============================]2000/2000 [==============================] 


Epoch 47/500
2000/2000 [==============================]2000/2000 [==============================] - 241s 120ms/step - loss: 1.3404 - acc: 0.5771 - val_loss: 1.8048 - val_acc: 0.4700

Epoch 48/500
2000/2000 [==============================]2000/2000 [==============================] - 239s 119ms/step - loss: 1.3324 - acc: 0.5782 - val_loss: 1.9565 - val_acc: 0.4437

Epoch 49/500
2000/2000 [==============================]2000/2000 [==============================] - 237s 118ms/step - loss: 1.3128 - acc: 0.5822 - val_loss: 1.8638 - val_acc: 0.4658

Epoch 50/500
2000/2000 [==============================]2000/2000 [==============================] - 238s 119ms/step - loss: 1.3155 - acc: 0.5843 - val_loss: 1.9561 - val_acc: 0.4520

Epoch 51/500
2000/2000 [==============================]2000/2000 [==============================] - 235s 117ms/step - loss: 1.3262 - acc: 0.5797 - val_loss: 1.8887 - val_acc: 0.4784

Epoch 52/500
2000/2000 [==============================]2000/2000 [==================


Epoch 71/500
2000/2000 [==============================]2000/2000 [==============================] - 240s 120ms/step - loss: 1.2865 - acc: 0.5935 - val_loss: 1.7901 - val_acc: 0.4806

Epoch 72/500
2000/2000 [==============================]2000/2000 [==============================] - 237s 118ms/step - loss: 1.2953 - acc: 0.5876 - val_loss: 1.8139 - val_acc: 0.4869

Epoch 73/500
2000/2000 [==============================]2000/2000 [==============================] - 239s 119ms/step - loss: 1.2985 - acc: 0.5880 - val_loss: 1.7594 - val_acc: 0.4856

Epoch 74/500
2000/2000 [==============================]2000/2000 [==============================] - 236s 118ms/step - loss: 1.2782 - acc: 0.5912 - val_loss: 1.7895 - val_acc: 0.4872

Epoch 75/500
2000/2000 [==============================]2000/2000 [==============================] - 239s 119ms/step - loss: 1.2762 - acc: 0.5916 - val_loss: 1.7766 - val_acc: 0.4881

Epoch 76/500
2000/2000 [==============================]2000/2000 [==================


Epoch 95/500
2000/2000 [==============================]2000/2000 [==============================] - 235s 118ms/step - loss: 1.2549 - acc: 0.5978 - val_loss: 1.7852 - val_acc: 0.4800

Epoch 96/500
2000/2000 [==============================]2000/2000 [==============================] - 234s 117ms/step - loss: 1.2640 - acc: 0.5954 - val_loss: 1.7581 - val_acc: 0.4845

Epoch 97/500
2000/2000 [==============================]2000/2000 [==============================] - 237s 118ms/step - loss: 1.2634 - acc: 0.5973 - val_loss: 1.7604 - val_acc: 0.4911

Epoch 98/500
2000/2000 [==============================]2000/2000 [==============================] - 238s 119ms/step - loss: 1.2763 - acc: 0.5933 - val_loss: 1.7578 - val_acc: 0.4941

Epoch 99/500
2000/2000 [==============================]2000/2000 [==============================] - 238s 119ms/step - loss: 1.2734 - acc: 0.5958 - val_loss: 1.7453 - val_acc: 0.4878

Epoch 100/500
2000/2000 [==============================]2000/2000 [=================

...


Epoch 310/500
2000/2000 [==============================]2000/2000 [==============================] - 227s 114ms/step - loss: 1.2619 - acc: 0.5972 - val_loss: 1.7343 - val_acc: 0.4938

Epoch 311/500
2000/2000 [==============================]2000/2000 [==============================] - 225s 113ms/step - loss: 1.2404 - acc: 0.6031 - val_loss: 1.7316 - val_acc: 0.4923

Epoch 312/500
2000/2000 [==============================]2000/2000 [==============================] - 226s 113ms/step - loss: 1.2454 - acc: 0.5991 - val_loss: 1.7266 - val_acc: 0.4947

Epoch 313/500
2000/2000 [==============================]2000/2000 [==============================] - 226s 113ms/step - loss: 1.2591 - acc: 0.5984 - val_loss: 1.7257 - val_acc: 0.4950

Epoch 314/500
2000/2000 [==============================]2000/2000 [==============================] - 226s 113ms/step - loss: 1.2433 - acc: 0.6006 - val_loss: 1.7357 - val_acc: 0.4950

Epoch 315/500
2000/2000 [==============================]2000/2000 [============


Epoch 334/500
2000/2000 [==============================]2000/2000 [==============================] - 234s 117ms/step - loss: 1.2528 - acc: 0.5973 - val_loss: 1.7282 - val_acc: 0.4925

Epoch 335/500
2000/2000 [==============================]2000/2000 [==============================] - 233s 117ms/step - loss: 1.2477 - acc: 0.6010 - val_loss: 1.7316 - val_acc: 0.4928

Epoch 336/500
2000/2000 [==============================]2000/2000 [==============================] - 232s 116ms/step - loss: 1.2629 - acc: 0.5976 - val_loss: 1.7356 - val_acc: 0.4944

Epoch 337/500
2000/2000 [==============================]2000/2000 [==============================] - 236s 118ms/step - loss: 1.2386 - acc: 0.6028 - val_loss: 1.7243 - val_acc: 0.4925

Epoch 338/500
2000/2000 [==============================]2000/2000 [==============================] - 236s 118ms/step - loss: 1.2500 - acc: 0.5985 - val_loss: 1.7306 - val_acc: 0.4936

Epoch 339/500
2000/2000 [==============================]2000/2000 [============


Epoch 358/500
2000/2000 [==============================]2000/2000 [==============================] - 231s 115ms/step - loss: 1.2473 - acc: 0.5987 - val_loss: 1.7335 - val_acc: 0.4920

Epoch 359/500
2000/2000 [==============================]2000/2000 [==============================] - 237s 118ms/step - loss: 1.2509 - acc: 0.5973 - val_loss: 1.7314 - val_acc: 0.4945

Epoch 360/500
2000/2000 [==============================]2000/2000 [==============================] - 238s 119ms/step - loss: 1.2568 - acc: 0.5958 - val_loss: 1.7312 - val_acc: 0.4927

Epoch 361/500
2000/2000 [==============================]2000/2000 [==============================] - 236s 118ms/step - loss: 1.2509 - acc: 0.6011 - val_loss: 1.7314 - val_acc: 0.4945

Epoch 362/500
2000/2000 [==============================]2000/2000 [==============================] - 232s 116ms/step - loss: 1.2545 - acc: 0.5993 - val_loss: 1.7328 - val_acc: 0.4959

Epoch 363/500
2000/2000 [==============================]2000/2000 [============


Epoch 382/500
2000/2000 [==============================]2000/2000 [==============================] - 237s 119ms/step - loss: 1.2617 - acc: 0.5962 - val_loss: 1.7310 - val_acc: 0.4964

Epoch 383/500
2000/2000 [==============================]2000/2000 [==============================] - 238s 119ms/step - loss: 1.2637 - acc: 0.5956 - val_loss: 1.7328 - val_acc: 0.4922

Epoch 384/500
2000/2000 [==============================]2000/2000 [==============================] - 239s 120ms/step - loss: 1.2458 - acc: 0.5980 - val_loss: 1.7310 - val_acc: 0.4945

Epoch 385/500
2000/2000 [==============================]2000/2000 [==============================] - 239s 119ms/step - loss: 1.2495 - acc: 0.5997 - val_loss: 1.7406 - val_acc: 0.4931

Epoch 386/500
2000/2000 [==============================]2000/2000 [==============================] - 238s 119ms/step - loss: 1.2570 - acc: 0.5965 - val_loss: 1.7338 - val_acc: 0.4947

Epoch 387/500
2000/2000 [==============================]2000/2000 [============


Epoch 406/500
2000/2000 [==============================]2000/2000 [==============================] - 237s 119ms/step - loss: 1.2524 - acc: 0.5997 - val_loss: 1.7308 - val_acc: 0.4945

Epoch 407/500
 822/2000 [===========>..................] 822/2000 [===========>..................] - ETA: 2:07 - loss: 1.2534 - acc: 0.5981